## Librerías

In [1]:
#Main libraries
import pandas as pd
import numpy as np
import os

#Utilities
from time import time
from statistics import mean 
from statistics import stdev

#Models
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier as RndFC
from sklearn.neural_network import MLPClassifier as ffp
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

#Dataset Preparation
from sklearn.model_selection import train_test_split

#Metrics
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn import metrics as met

#Cross validation
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

## Funciones

In [2]:
# UP-Fall:
def create_df(window, frame, method, counter, total_time, metric, values):
    df = pd.DataFrame({
            'Method':[method],
            'Counter':[counter],
            'Time':[total_time],
            'Metric':[metric]
    })
    for i in range (0,len(values)):
        df[str(i)] = [values[i]]
        
    return df

In [3]:
def changeLabels(l):
    l.loc[(l[0] == 1.0),'label'] = 'Fall'
    l.loc[(l[0] == 2.0),'label'] = 'Fall'
    l.loc[(l[0] == 3.0),'label'] = 'Fall'
    l.loc[(l[0] == 4.0),'label'] = 'Fall'
    l.loc[(l[0] == 5.0),'label'] = 'Fall'
    l.loc[(l[0] == 11.0),'label'] = 'Fall'

    l.loc[(l[0] == 6.0),'label'] = 'Not Fall'
    l.loc[(l[0] == 7.0),'label'] = 'Not Fall'
    l.loc[(l[0] == 8.0),'label'] = 'Not Fall'
    l.loc[(l[0] == 9.0),'label'] = 'Not Fall'
    l.loc[(l[0] == 10.0),'label'] = 'Not Fall'
    l.loc[(l[0] == 20.0),'label'] = 'Not Fall'

## Ejecución

In [ ]:
base_dir = '/Users/heilym/UP-fall/'
output_dir = ''
windows = ['2'] #0,1,2
features_inputfile = '/Users/heilym/UP-fall/Features.csv'
labels_inputfile = '/Users/heilym/UP-fall/Labels.csv'
metrics_outfile = "Metrics1.csv"
confusion_matrix_outfile = "Confusion_matrix1.csv"
recall_array = {}
metric_files = False

classifiers = {
    #'RF': RndFC(n_estimators=100),
    #'KNN': KNN(),
    'SVM': svm.SVC(C=1000, gamma=0.0001),
    #'MLP': ffp(),
    #'AdaBoost': AdaBoostClassifier(DecisionTreeClassifier(max_depth=15, min_samples_split=20), n_estimators=10,learning_rate=1),
}

## Iteración por Ventana
for window in windows:
    
    print()
    print("Ventana: {}".format(window))

    ##Obtener Frames desde archivos
    #frames_by_window = [3]
    frame = 3
    '''
    for file in os.listdir("'/Users/heilym/UP-fall/'V2".format(base_dir,window)):
        if file.startswith("MatrixFeatures"):
            frames_by_window.append(file.split("MatrixFeatures")[1].split(".")[0])
    '''
    ## Iteración para cada frame por ventana
    #for frame in frames_by_window:
        
    ## Leer Archivos
    features = pd.read_csv(features_inputfile,header=None)
    labels = pd.read_csv(labels_inputfile,header=None)
    changeLabels(labels)

    #Imprimir etiquetas y dimensiones
    print()
    print("Frame: {}".format(frame))
    print("Etiquetas:")
    print(labels['label'].value_counts())
    print("Dimensiones:")
    print (features.shape, labels.shape)

    ##Iteración de clasificadores seleccionados
    for method in classifiers:

        print()
        print('Method: {}'.format(method))
        classifier = classifiers[method]
        counter = 0

        #K-fold Cross-Validation
        print('K-Fold Cross Validation')
        start_time = time()
        cv = KFold(n_splits=10, random_state=42, shuffle=True)
        for train_index, test_index in cv.split(features):
            start_time = time()
            counter += 1
            print('K-Fold - Counter: %d' % (counter))

            X_train_kf, X_test_kf, y_train_kf, y_test_kf = features.iloc[train_index], features.iloc[test_index], labels['label'].iloc[train_index], labels['label'].iloc[test_index]
            classifier.fit(X_train_kf, y_train_kf)
            estimates_kf = classifier.predict(X_test_kf)

            # K-Fold KPI
            precision_kf, recall_kf, fscore_kf, support_kf = score(y_test_kf, estimates_kf)
            #print('K-Fold - Precision: {}'.format(precision_kf))
            #print('K-Fold - Recall: {}'.format(recall_kf))
            #print('K-Fold - Fscore: {}'.format(fscore_kf))
            #print('K-Fold - Support: {}'.format(support_kf))

            #Save KPIs
            if metric_files:
                header = True
                if os.path.exists(metrics_outfile.format(output_dir,window,frame)):
                    header = False
                create_df(window,frame,method,counter,(time() - start_time),'Precision',precision_kf).to_csv(metrics_outfile.format(output_dir+"/V"+window,frame), mode='a',header=header,index=False,sep=';')
                header = False
                create_df(window,frame,method,counter,(time() - start_time),'Recall',recall_kf).to_csv(metrics_outfile.format(output_dir+"/V"+window,frame), mode='a',header=header,index=False,sep=';')
                create_df(window,frame,method,counter,(time() - start_time),'Fscore',fscore_kf).to_csv(metrics_outfile.format(output_dir+"/V"+window,frame), mode='a',header=header,index=False,sep=';')
                create_df(window,frame,method,counter,(time() - start_time),'Support',support_kf).to_csv(metrics_outfile.format(output_dir+"/V"+window,frame), mode='a',header=header,index=False,sep=';')
                pd.crosstab(np.array(y_test_kf).flatten(), estimates_kf).to_csv(confusion_matrix_outfile.format(output_dir+"/V"+window,method,frame), mode='a',header=header,index=False,sep=';')
            else:
                header = True
                if os.path.exists(metrics_outfile.format(output_dir,'_todos','_todos')):
                    header = False
                create_df(window,frame,method,counter,(time() - start_time),'Precision',precision_kf).to_csv(metrics_outfile.format(output_dir,'_todos','_todos'), mode='a',header=header,index=False,sep=';')
                header = False
                create_df(window,frame,method,counter,(time() - start_time),'Recall',recall_kf).to_csv(metrics_outfile.format(output_dir,'_todos','_todos'), mode='a',header=header,index=False,sep=';')
                create_df(window,frame,method,counter,(time() - start_time),'Fscore',fscore_kf).to_csv(metrics_outfile.format(output_dir,'_todos','_todos'), mode='a',header=header,index=False,sep=';')
                create_df(window,frame,method,counter,(time() - start_time),'Support',support_kf).to_csv(metrics_outfile.format(output_dir,'_todos','_todos'), mode='a',header=header,index=False,sep=';')
                pd.crosstab(np.array(y_test_kf).flatten(), estimates_kf).to_csv(confusion_matrix_outfile.format(output_dir,'Todos','Todos','_todos'), mode='a',header=header,index=False,sep=';')
                    
print("-"*50)



Ventana: 2

Frame: 3
Etiquetas:
Not Fall    160056
Fall         43469
Name: label, dtype: int64
Dimensiones:
(203525, 153) (203525, 2)

Method: SVM
K-Fold Cross Validation
K-Fold - Counter: 1
K-Fold - Counter: 2
K-Fold - Counter: 3
K-Fold - Counter: 4


In [ ]:
print (labels.shape)